In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.plotting import andrews_curves
from statsmodels.nonparametric.smoothers_lowess import lowess
red_wine=pd.read_csv("winequality-red.csv",sep=";")
white_wine=pd.read_csv("winequality-white.csv",sep=";")
red_wine.columns=red_wine.columns.str.replace(" ","_")
white_wine.columns=white_wine.columns.str.replace(" ","_")
red_wine["type"]="red"
white_wine["type"]="white"
red_wine["total_acid"]=(red_wine["fixed_acidity"]+red_wine["volatile_acidity"]+red_wine["citric_acid"])
white_wine["total_acid"]=white_wine["fixed_acidity"]+white_wine["volatile_acidity"]+white_wine["citric_acid"]
red_wine["quality_label"] = red_wine["quality"].apply(lambda value: "low"
if value <= 5 else "medium"
if value <= 7 else "high")
white_wine["quality_label"]=white_wine["quality"].apply(lambda value: "low"
if value<=5 else "medium"
if value<=7 else "high")
red_wine["quality_label"]=pd.Categorical(red_wine["quality_label"],categories=["low","medium","high"])
white_wine["quality_label"]=pd.Categorical(white_wine["quality_label"],categories=["low","medium","high"])

In [ ]:
def plotter(a,b):
    fig, axes = plt.subplots(1,2, figsize=(10,6),facecolor="snow")
    x=sns.regplot(ax=axes[0],x=red_wine[a],y=red_wine[b],lowess=True,scatter_kws={"color":"crimson"},line_kws={"color":"black"}).set(title="Red wines")
    y=sns.regplot(ax=axes[1],x=white_wine[a],y=white_wine[b],lowess=True,scatter_kws={"color":"crimson"},line_kws={"color":"black"}).set(title="White wines")
    return x,y

In [ ]:
def boxplotter(x,y,z,winetype):
    fig, axes = plt.subplots(1,2, figsize=(10,6),facecolor="snow")
    if len(winetype)==1599:
        title="Red Wines"
    else:
        title="White Wines"
    temp=winetype[[z,x]]
    temp.boxplot(by=z, ax=axes[0],showfliers=True,notch=True,patch_artist=True,color="dimgray");axes[0].set_title(""); axes[0].set_xlabel(z); axes[0].set_ylabel(x)
    temp=winetype[[z,y]]
    temp.boxplot(by=z, ax=axes[1],showfliers=False,notch=True,patch_artist=True,color="dimgray");axes[1].set_title(""); axes[1].set_xlabel(z); axes[1].set_ylabel(y)
    fig.suptitle(title)

In [ ]:
con1=red_wine[["residual_sugar","total_sulfur_dioxide","sulphates","alcohol","volatile_acidity","quality"]]
con2=white_wine[["residual_sugar","total_sulfur_dioxide","sulphates","alcohol","volatile_acidity","quality"]]

In [ ]:
corr=con1.corr()
corr.style.background_gradient(cmap="coolwarm")

PCA analysis for wine quality

In [53]:
wines=pd.concat([white_wine])
wines=wines.sample(frac=1,random_state=42).reset_index(drop=True)
names=["residual_sugar","total_sulfur_dioxide","sulphates","alcohol","volatile_acidity","type","quality"]
features=["residual_sugar","total_sulfur_dioxide","sulphates","alcohol","volatile_acidity","type"]
temp=wines
temp.replace({"white":0,"red":1},inplace=True)
temp.replace({"low":0,"medium":1,"high":2},inplace=True)

In [80]:
pca=PCA()
components=pca.fit_transform(temp[features])
labels={str(i): f"PC {i+1} ({var:.1f}%)"
        for i, var in enumerate(pca.explained_variance_ratio_*100)}
fig=px.scatter_matrix(components,labels=labels,dimensions=range(2),color=temp["quality"])
fig.update_traces(diagonal_visible=False)
fig.show()

In [46]:
red_wine["citric_acid"].describe()

count    1599.000000
mean        0.270976
std         0.194801
min         0.000000
25%         0.090000
50%         0.260000
75%         0.420000
max         1.000000
Name: citric_acid, dtype: float64

In [58]:
white_wine["quality"].unique()

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)